In [9]:
import pandas as pd
import numpy as np

In [10]:
spotify = pd.read_csv('../../Data/Processed/spotify_data.csv')
deezer = pd.read_csv('../../Data/Raw/deezer_data_v.csv')

C:\Users\vojte\AppData\Local\Temp\ipykernel_16888\697030822.py:2: DtypeWarning: Columns (0,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  deezer = pd.read_csv('../../Data/Raw/deezer_data_v.csv')


In [11]:
deezer = deezer.dropna()

In [12]:
deezer['id'] = deezer['id'].astype(str)
deezer['duration'] = pd.to_numeric(deezer['duration'], errors='coerce')  # Convert to numeric, invalid values become NaN
deezer['duration'] = deezer['duration'].fillna(0).astype(float)  # Replace NaN with 0 and convert to float
deezer['rank'] = pd.to_numeric(deezer['rank'], errors='coerce')  # Convert to numeric, invalid values become NaN
deezer['rank'] = deezer['rank'].fillna(0).astype(int)  # Replace NaN with 0 and convert to int

In [ ]:
# merged_df = pd.merge(spotify.add_prefix('spotify.'), deezer.add_prefix('deezer.'), left_on='spotify.external_ids.isrc', right_on='deezer.isrc', how='inner')
merged_df = pd.merge(
    spotify.add_prefix('spotify.'), 
    deezer.add_prefix('deezer.'), 
    left_on=['spotify.external_links.isrc'], 
    right_on=['deezer.isrc'], 
    how='inner'
)
print(merged_df)

                                   spotify.name  \
0                              Die With A Smile   
1                            BIRDS OF A FEATHER   
2                                          DtMF   
3                                That’s So True   
4       Clean Baby Sleep White Noise (Loopable)   
...                                         ...   
139753                                       旅行   
139754                       Chacoalha a Rabeta   
139755                               Скажи мені   
139756          All I Want For Christmas Is You   
139757                  Dá-Lhe, Dá-Lhe Rapariga   

                                       spotify.album.name  \
0                                        Die With A Smile   
1                                    HIT ME HARD AND SOFT   
2                                    DeBÍ TiRAR MáS FOToS   
3                               The Secret of Us (Deluxe)   
4                      Best White Noise For Sleeping Baby   
...                  

In [14]:
merged_df = merged_df.sort_values('deezer.rank', ascending=False).drop_duplicates(subset='spotify.external_ids.isrc', keep='first')
merged_df = merged_df[merged_df['deezer.rank'] != 0]
merged_df['deezer.rank'] = merged_df['deezer.rank'] / 10000

In [15]:
merged_df.describe(include='all')

,spotify.name,spotify.album.name,spotify.duration_ms,spotify.popularity,spotify.id,spotify.external_ids.isrc,spotify.album.release_date_precision,spotify.album.release_date,spotify.album.total_tracks,spotify.disc_number,...,deezer.id,deezer.title,deezer.artist,deezer.album,deezer.duration,deezer.link,deezer.rank,deezer.release_date,deezer.explicit_lyrics,deezer.isrc
count,139215,139215,1.392150e+05,139215.000000,139215,139215,139215,139215,139215.000000,139215.000000,...,139215,139215,139215,139215,139215.000000,139215,139215.000000,139215,139215,139215
unique,118994,72455,NaN,NaN,139215,139215,3,7567,NaN,NaN,...,139168,118994,27501,81417,NaN,139168,NaN,7432,4,139168
top,Intro,7,NaN,NaN,6IcsbETuviVu6UTiBTcxY4,FR9W12422486,day,2024-11-29,NaN,NaN,...,1605214892.0,Intro,Jul,Greatest Hits,NaN,https://www.deezer.com/track/1605214892,NaN,2023-10-27,False,TWCC32123658
freq,134,68,NaN,NaN,1,1,137218,292,NaN,NaN,...,2,134,466,72,NaN,2,NaN,255,59523,2
mean,NaN,NaN,1.980455e+05,39.936903,NaN,NaN,NaN,NaN,9.600754,1.008943,...,NaN,NaN,NaN,NaN,197.579729,NaN,22.062449,NaN,NaN,NaN
std,NaN,NaN,6.789057e+04,17.371477,NaN,NaN,NaN,NaN,9.482431,0.114523,...,NaN,NaN,NaN,NaN,68.119928,NaN,19.839126,NaN,NaN,NaN
min,NaN,NaN,3.000000e+04,1.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,0.000100,NaN,NaN,NaN
25%,NaN,NaN,1.650000e+05,28.000000,NaN,NaN,NaN,NaN,1.000000,1.000000,...,NaN,NaN,NaN,NaN,165.000000,NaN,6.062050,NaN,NaN,NaN
50%,NaN,NaN,1.920000e+05,40.000000,NaN,NaN,NaN,NaN,10.000000,1.000000,...,NaN,NaN,NaN,NaN,192.000000,NaN,16.455600,NaN,NaN,NaN
75%,NaN,NaN,2.221540e+05,52.000000,NaN,NaN,NaN,NaN,14.000000,1.000000,...,NaN,NaN,NaN,NaN,222.000000,NaN,32.822850,NaN,NaN,NaN


In [16]:
merged_df.to_csv('../../Data/Processed/spotify_deezer_merged.csv', index=False)